In [1]:
from google.cloud import bigquery

client = bigquery.Client()

Using Kaggle's public dataset BigQuery integration.


In [2]:
import pandas as pd

df_genes = pd.read_csv('https://raw.githubusercontent.com/marcin119a/data/refs/heads/main/joined.csv')

In [ ]:
df_genes

In [6]:
genes = "', '".join(list(df_genes['Hugo Symbol'].unique()))

In [7]:
dataset = 'isb-cgc-bq.annotations.methylation_annotation_hg38_gdc_current'

query = f"""
SELECT DISTINCT
  g.symbol AS gene_symbol,
  CpG_probe_id,
  chromosome,
  position
FROM `{dataset}`,
UNNEST(genes) AS g
WHERE g.symbol IN ('{genes}')
ORDER BY g.symbol
"""

query_job = client.query(query)
probles_for_onkodb = query_job.to_dataframe()


/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [8]:
import pandas as pd 

df_metadata = pd.read_csv('https://raw.githubusercontent.com/marcin119a/data/refs/heads/main/bq-metadata.csv')

In [9]:
df_hg38_methylation = df_metadata[(df_metadata['Full ID'].str.contains(r'DNA_methylation.*hg38', regex=True)) & (df_metadata['BQ Dataset']=='TCGA') ]


In [ ]:
import re
import pandas as pd
from google.cloud import bigquery

client = bigquery.Client()



project_id = "isb-cgc-bq"
dataset_id = "TCGA"

tables = df_hg38_methylation['BQ Table'].tolist()
for table_name in tables[::-1]:
    # Wyciągnij numer/oznaczenie chromosomu z nazwy tabeli
    m = re.search(r'chr(\w+)_hg38', table_name)
    if not m:
        continue
    chrom = m.group(1)  # np. "1", "2", ..., "X", "Y"

    # Filtrowanie probes dla tego chromosomu
    probes_for_chromosome = probles_for_onkodb[probles_for_onkodb['chromosome'].astype(str) == ('chr'+chrom)]
    CpG_probe_ids = ", ".join([f"'{p}'" for p in probes_for_chromosome['CpG_probe_id'].unique()])

    full_table = f"`{project_id}.{dataset_id}.{table_name}`"
    print(f"Pobieram dane z: {full_table} (chromosom {chrom})...")

    query = f"""
    SELECT 
      case_barcode,
      ARRAY_AGG(probe_id ORDER BY probe_id) AS probe_id_ids,
      ARRAY_AGG(beta_value ORDER BY probe_id) AS beta_values
    FROM {full_table}
    WHERE 
      probe_id IN ({CpG_probe_ids})
    GROUP BY case_barcode
    """

    try:
        df = client.query(query).to_dataframe()

        if not df.empty:
            output_path = f"chr{chrom}.parquet"
            df.to_parquet(output_path, index=False)
            print(f"Zapisano dane dla chromosomu {chrom} do pliku {output_path}")
        else:
            print(f"Brak danych w {table_name} dla chromosomu {chrom}")

        del df  # zwolnienie pamięci

    except Exception as e:
        print(f"Błąd w tabeli {table_name}: {e}")

Using Kaggle's public dataset BigQuery integration.
Pobieram dane z: `isb-cgc-bq.TCGA.DNA_methylation_chrY_hg38_gdc_current` (chromosom Y)...


/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Zapisano dane dla chromosomu Y do pliku chrY.parquet
Pobieram dane z: `isb-cgc-bq.TCGA.DNA_methylation_chrX_hg38_gdc_current` (chromosom X)...


/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Zapisano dane dla chromosomu X do pliku chrX.parquet
Pobieram dane z: `isb-cgc-bq.TCGA.DNA_methylation_chr9_hg38_gdc_current` (chromosom 9)...


...

/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Zapisano dane dla chromosomu 20 do pliku chr20.parquet
Pobieram dane z: `isb-cgc-bq.TCGA.DNA_methylation_chr2_hg38_gdc_current` (chromosom 2)...


/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Zapisano dane dla chromosomu 2 do pliku chr2.parquet
Pobieram dane z: `isb-cgc-bq.TCGA.DNA_methylation_chr19_hg38_gdc_current` (chromosom 19)...


/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
